# Forest Type Mapping Model Implementation

## 1. Import Libraries

In [24]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
import time
import sys

In [25]:
# Read the CSV file without setting any column as the index
df = pd.read_csv("C:/Users/vedan/OneDrive/Desktop/ForestTypeMapping/forest-type-mapping-ESP32/Data/training.csv")
df.head()

,class,b1,b2,b3,b4,b5,b6,b7,b8,b9,...,pred_minus_obs_H_b9,pred_minus_obs_S_b1,pred_minus_obs_S_b2,pred_minus_obs_S_b3,pred_minus_obs_S_b4,pred_minus_obs_S_b5,pred_minus_obs_S_b6,pred_minus_obs_S_b7,pred_minus_obs_S_b8,pred_minus_obs_S_b9
0,d,39,36,57,91,59,101,93,27,60,...,-2.36,-18.41,-1.88,-6.43,-21.03,-1.60,-6.18,-22.50,-5.20,-7.86
1,h,84,30,57,112,51,98,92,26,62,...,-2.26,-16.27,-1.95,-6.25,-18.79,-1.99,-6.18,-23.41,-8.87,-10.83
2,s,53,25,49,99,51,93,84,26,58,...,-1.46,-15.92,-1.79,-4.64,-17.73,-0.48,-4.69,-19.97,-4.10,-7.07
3,s,59,26,49,103,47,92,82,25,56,...,2.68,-13.77,-2.53,-6.34,-22.03,-2.34,-6.60,-27.10,-7.99,-10.81
4,d,57,49,66,103,64,106,114,28,59,...,-2.94,-21.74,-1.64,-4.62,-23.74,-0.85,-5.50,-22.83,-2.74,-5.84


In [26]:
df.columns

Index(['class', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9',
       'pred_minus_obs_H_b1', 'pred_minus_obs_H_b2', 'pred_minus_obs_H_b3',
       'pred_minus_obs_H_b4', 'pred_minus_obs_H_b5', 'pred_minus_obs_H_b6',
       'pred_minus_obs_H_b7', 'pred_minus_obs_H_b8', 'pred_minus_obs_H_b9',
       'pred_minus_obs_S_b1', 'pred_minus_obs_S_b2', 'pred_minus_obs_S_b3',
       'pred_minus_obs_S_b4', 'pred_minus_obs_S_b5', 'pred_minus_obs_S_b6',
       'pred_minus_obs_S_b7', 'pred_minus_obs_S_b8', 'pred_minus_obs_S_b9'],
      dtype='object')

In [27]:
print("Unique values in 'class' column:", df['class'].unique())

Unique values in 'class' column: ['d ' 'h ' 's ' 'o ']


In [28]:
# Get basic statistics for numerical columns
print("\nSummary statistics for numerical columns:")
df.describe()


Summary statistics for numerical columns:


,b1,b2,b3,b4,b5,b6,b7,b8,b9,pred_minus_obs_H_b1,...,pred_minus_obs_H_b9,pred_minus_obs_S_b1,pred_minus_obs_S_b2,pred_minus_obs_S_b3,pred_minus_obs_S_b4,pred_minus_obs_S_b5,pred_minus_obs_S_b6,pred_minus_obs_S_b7,pred_minus_obs_S_b8,pred_minus_obs_S_b9
count,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,...,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000
mean,62.949495,41.020202,63.676768,101.409091,58.732323,100.651515,90.601010,28.691919,61.116162,50.818889,...,-5.594141,-20.037576,-1.007121,-4.355657,-20.996919,-0.973737,-4.597626,-18.840000,-1.570808,-4.155859
std,12.779563,17.832543,17.314545,14.804627,12.392648,11.190314,15.588861,8.977752,9.787158,12.842321,...,9.769193,4.948562,1.783671,2.352311,6.490763,0.702619,1.736712,5.251095,1.807792,1.982423
min,34.000000,25.000000,47.000000,54.000000,44.000000,84.000000,54.000000,21.000000,50.000000,7.660000,...,-53.530000,-32.950000,-8.800000,-11.210000,-40.370000,-3.270000,-8.730000,-34.140000,-8.870000,-10.830000
25%,54.000000,28.000000,52.000000,92.250000,49.000000,92.000000,80.000000,24.000000,55.000000,40.667500,...,-6.627500,-23.325000,-1.860000,-5.790000,-24.090000,-1.290000,-5.747500,-22.237500,-2.370000,-5.122500
50%,60.000000,31.500000,57.000000,99.500000,55.000000,98.000000,91.000000,25.000000,58.000000,53.030000,...,-2.255000,-20.020000,-0.970000,-4.350000,-20.465000,-0.945000,-4.540000,-19.200000,-1.420000,-4.125000
75%,70.750000,50.750000,69.000000,111.750000,65.000000,107.000000,101.000000,27.000000,63.000000,59.920000,...,0.247500,-17.787500,-0.042500,-2.882500,-17.955000,-0.642500,-3.617500,-16.227500,-0.655000,-3.105000
max,105.000000,160.000000,196.000000,172.000000,98.000000,136.000000,139.000000,82.000000,109.000000,83.320000,...,5.740000,5.130000,12.460000,7.370000,1.880000,3.440000,3.940000,3.670000,8.840000,7.790000


In [29]:
# Get the unique values in each column to see if there are categorical data
print("\nUnique values in each column:")
for column in df.columns:
    print(f"{column}: {df[column].nunique()} unique values")


Unique values in each column:
class: 4 unique values
b1: 50 unique values
b2: 52 unique values
b3: 50 unique values
b4: 58 unique values
b5: 47 unique values
b6: 42 unique values
b7: 61 unique values
b8: 32 unique values
b9: 38 unique values
pred_minus_obs_H_b1: 191 unique values
pred_minus_obs_H_b2: 189 unique values
pred_minus_obs_H_b3: 191 unique values
pred_minus_obs_H_b4: 193 unique values
pred_minus_obs_H_b5: 189 unique values
pred_minus_obs_H_b6: 194 unique values
pred_minus_obs_H_b7: 195 unique values
pred_minus_obs_H_b8: 183 unique values
pred_minus_obs_H_b9: 193 unique values
pred_minus_obs_S_b1: 183 unique values
pred_minus_obs_S_b2: 165 unique values
pred_minus_obs_S_b3: 173 unique values
pred_minus_obs_S_b4: 185 unique values
pred_minus_obs_S_b5: 123 unique values
pred_minus_obs_S_b6: 173 unique values
pred_minus_obs_S_b7: 182 unique values
pred_minus_obs_S_b8: 166 unique values
pred_minus_obs_S_b9: 166 unique values


In [30]:
# Check the distribution of the 'class' column
print("\nClass distribution:")
df['class'].value_counts()


Class distribution:


class
s     59
d     54
h     48
o     37
Name: count, dtype: int64

## 2. Load the datasets

In [32]:
train_df = pd.read_csv("C:/Users/vedan/OneDrive/Desktop/ForestTypeMapping/forest-type-mapping-ESP32/Data/training.csv")
test_df = pd.read_csv("C:/Users/vedan/OneDrive/Desktop/ForestTypeMapping/forest-type-mapping-ESP32/Data/testing.csv")

## 3. Data Preprocessing

### 3.1 Encode the Target Variable

In [35]:
# Encode the 'class' column
label_encoder = LabelEncoder()
train_df['class'] = label_encoder.fit_transform(train_df['class'].str.strip())

### 3.2 Separate Features and Target Variable

In [37]:
# Separate features and target variable in training data
columns_to_drop = ['class', 'pred_minus_obs_H_b1', 'pred_minus_obs_H_b2', 'pred_minus_obs_H_b3',
       'pred_minus_obs_H_b4', 'pred_minus_obs_H_b5', 'pred_minus_obs_H_b6',
       'pred_minus_obs_H_b7', 'pred_minus_obs_H_b8', 'pred_minus_obs_H_b9',
       'pred_minus_obs_S_b1', 'pred_minus_obs_S_b2', 'pred_minus_obs_S_b3',
       'pred_minus_obs_S_b4', 'pred_minus_obs_S_b5', 'pred_minus_obs_S_b6',
       'pred_minus_obs_S_b7', 'pred_minus_obs_S_b8', 'pred_minus_obs_S_b9']

X = train_df.drop(columns=columns_to_drop)
y = train_df['class']

### 3.3 One-Hot Encoding and Normalization

In [39]:
# One-hot encoding the target variable
y = to_categorical(y)

# Normalize the feature data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### 3.4 Train-Validation Split

In [41]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.20, random_state=42)

## 4. Model Configuration

In [43]:
# Define model parameters
DENSE1_SIZE = 16 # Increased size for more capacity
DENSE2_SIZE = 8
DENSE3_SIZE = 4
NUM_OF_EPOCHS = 100  # Increased epochs for better training
BATCH_SIZE = 16  # Adjusted batch size

## 5. Build the Model

In [45]:
# Build the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(DENSE1_SIZE, activation='relu'))
# model.add(tf.keras.layers.Dropout(0.3))  # Added dropout for regularization
model.add(tf.keras.layers.Dense(DENSE2_SIZE, activation='relu'))
model.add(tf.keras.layers.Dense(DENSE3_SIZE, activation='relu'))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))

C:\Users\vedan\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [46]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 9)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 4)                   │              36 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4)                   │              20 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 352 (1.38 KB)

 Trainable params: 352 (1.38 KB)

 Non-trainable params: 0 (0.00 B)

None


## 6. Train the Model

In [48]:
# Train the model
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NUM_OF_EPOCHS, 
                    verbose=1, validation_data=(X_val, y_val))

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.2914 - loss: 1.4384 - val_accuracy: 0.4250 - val_loss: 1.3926
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3942 - loss: 1.4017 - val_accuracy: 0.3750 - val_loss: 1.3311
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4155 - loss: 1.3344 - val_accuracy: 0.5000 - val_loss: 1.2769
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4935 - loss: 1.2769 - val_accuracy: 0.6500 - val_loss: 1.2392
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6216 - loss: 1.2487 - val_accuracy: 0.6750 - val_loss: 1.2001
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5921 - loss: 1.1946 - val_accuracy: 0.7250 - val_loss: 1.1526
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6985 - loss: 1.1372 - val_accuracy: 0.7250 - val_loss: 1.1017
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7249 - loss: 1.0812 - val_accuracy: 0.80

## 7. Evaluate the Model

In [50]:
# Evaluate the model on the training data
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

# Evaluate the model on the validation (testing) data
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation (Testing) Accuracy: {val_accuracy * 100:.2f}%")

Training Accuracy: 96.20%
Validation (Testing) Accuracy: 95.00%


In [51]:
# Ensure labels are consistent by applying LabelEncoder to known classes only
test_df['class'] = label_encoder.transform(test_df['class'].str.strip())

# Separate features and target variable in testing data
X_test = test_df.drop(columns=columns_to_drop)
y_test = test_df['class']

# One-hot encode the target variable in testing data
y_test = to_categorical(y_test)

# Normalize the feature data in the testing set
X_test_scaled = scaler.transform(X_test)

# Evaluate the model on the separate testing data
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Testing Accuracy: {test_accuracy * 100:.2f}%")


Testing Accuracy: 80.92%


## 8. Model Conversion to TensorFlow Lite

In [53]:
def representative_dataset():
    for _ in range(100):
        data = X_test_scaled.astype(np.float32)
        yield [data]

print(representative_dataset())


<generator object representative_dataset at 0x000001CB910D6190>


In [54]:
# Save the model
model.save('ForestTypeMappingModel.h5')

In [55]:
# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Apply optimizations (optional)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Representative dataset for quantization (provide a function for this)
def representative_dataset():
    for _ in range(100):
        yield [X_train.astype(np.float32)]  # Use training data or any relevant subset

converter.representative_dataset = representative_dataset

# Convert the model
tflite_model = converter.convert()

# Save the converted model
with open('ForestTypeMappingModel.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model has been converted and saved as 'ForestTypeMappingModel.tflite'.")

INFO:tensorflow:Assets written to: C:\Users\vedan\AppData\Local\Temp\tmpvumh3pdb\assets


INFO:tensorflow:Assets written to: C:\Users\vedan\AppData\Local\Temp\tmpvumh3pdb\assets


Saved artifact at 'C:\Users\vedan\AppData\Local\Temp\tmpvumh3pdb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  1973823267152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1973823266384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1973823266576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1973823266000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1973823269072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1973823269648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1973823268496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1973823270800: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model has been converted and saved as 'ForestTypeMappingModel.tflite'.


C:\Users\vedan\AppData\Roaming\Python\Python312\site-packages\tensorflow\lite\python\convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


## 9. Load and Test TFLite Model

In [57]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="ForestTypeMappingModel.tflite")
interpreter.allocate_tensors()

# Get input and output tensors details.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print('Input details:\n', input_details)
print('Output details:\n', output_details)

Input details:
 [{'name': 'serving_default_keras_tensor:0', 'index': 0, 'shape': array([1, 9]), 'shape_signature': array([-1,  9]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details:
 [{'name': 'StatefulPartitionedCall_1:0', 'index': 17, 'shape': array([1, 4]), 'shape_signature': array([-1,  4]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [58]:
# Test the model on random input data.
input_shape = input_details[0]['shape']
random_input_data = np.random.random_sample(input_shape).astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], random_input_data)

interpreter.invoke()

# Get the output tensor.
random_output_data = interpreter.get_tensor(output_details[0]['index'])
print("Random Input Inference Output:")
print(random_output_data)

# Verify if the same random input data is given to the original model, what is the output
keras_output = model.predict(random_input_data)
print("Keras Model Prediction Output for Random Input:")
print(keras_output)

Random Input Inference Output:
[[0.015625   0.015625   0.96484375 0.00390625]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
Keras Model Prediction Output for Random Input:
[[0.01600724 0.01638388 0.96319747 0.00441144]]


In [59]:
# Suppose scaler is already fitted on the training data
scaler = StandardScaler()
scaler.fit(X)  # Where X is the feature set from the training data

# Sample input
input_data = np.array([51 , 57  ,77 ,  90  ,89  ,123 ,  97 , 47,  83]).reshape(1, -1)

# Scale the input
scaled_input = scaler.transform(input_data)
print("Scaled Input:", scaled_input)


Scaled Input: [[-0.93741743  0.89837465  0.77143262 -0.77259708  2.44858083  2.00219009
   0.41152528  2.04444149  2.24164257]]


C:\Users\vedan\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [60]:
# Assume we have the label encoder used during training
label_encoder = LabelEncoder()
label_encoder.classes_ = np.array(['d','h','o','s'])  # Replace with your actual class names

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="ForestTypeMappingModel.tflite")
interpreter.allocate_tensors()

# Get input and output tensors details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [61]:
# Define a sample input (e.g., one row of normalized test data)
# Adjust values as needed for meaningful test cases
specific_input_data = np.array([[-1.8787969 , -0.28223274, -0.3865936 , -0.7048794, 0.02165439 , 0.03122059,
   0.15428138 ,-0.18893459 ,-0.11433257]], dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], specific_input_data)

# Run inference
interpreter.invoke()

# Get the output tensor (predicted probabilities)
output_data = interpreter.get_tensor(output_details[0]['index'])[0]
print("Model Output Probabilities:", output_data)

# Get the index of the highest probability, which corresponds to the predicted class
predicted_class_index = np.argmax(output_data)

# Decode the class index back to the original class label
predicted_class_label = label_encoder.inverse_transform([predicted_class_index])[0]
print("Predicted Forest Class:", predicted_class_label)


Model Output Probabilities: [0.9765625  0.         0.00390625 0.01953125]
Predicted Forest Class: d


In [62]:
# Define a sample input (e.g., one row of normalized test data)
# Adjust values as needed for meaningful test cases
specific_input_data = np.array([[ 1.72982439, -0.73198794, -0.44449491,  1.25893309 ,-0.54462844 ,-0.23754798
  , 0.66876918, -0.18893459,  0.39783594]], dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], specific_input_data)

# Run inference
interpreter.invoke()

# Get the output tensor (predicted probabilities)
output_data = interpreter.get_tensor(output_details[0]['index'])[0]
print("Model Output Probabilities:", output_data)

# Get the index of the highest probability, which corresponds to the predicted class
predicted_class_index = np.argmax(output_data)

# Decode the class index back to the original class label
predicted_class_label = label_encoder.inverse_transform([predicted_class_index])[0]
print("Predicted Forest Class:", predicted_class_label)


Model Output Probabilities: [0.00390625 0.97265625 0.         0.0234375 ]
Predicted Forest Class: h


In [63]:
# Define a sample input (e.g., one row of normalized test data)
# Adjust values as needed for meaningful test cases
specific_input_data = np.array([[-0.54517599 ,-0.67576854, -0.79190278, -0.56944406, -0.62552599, -0.59590607,
  -0.87469423, -0.3006034 , -0.31919997]], dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], specific_input_data)

# Run inference
interpreter.invoke()

# Get the output tensor (predicted probabilities)
output_data = interpreter.get_tensor(output_details[0]['index'])[0]
print("Model Output Probabilities:", output_data)

# Get the index of the highest probability, which corresponds to the predicted class
predicted_class_index = np.argmax(output_data)

# Decode the class index back to the original class label
predicted_class_label = label_encoder.inverse_transform([predicted_class_index])[0]
print("Predicted Forest Class:", predicted_class_label)


Model Output Probabilities: [0.      0.03125 0.      0.96875]
Predicted Forest Class: s


In [64]:
# Define the specific input data
specific_input_data = np.array([[-0.93741743, 0.89837465, 0.77143262, -0.77259708, 
                                 2.44858083, 2.00219009, 0.41152528, 2.04444149, 
                                 2.24164257]], dtype=np.float32)

# Predict with the original Keras model
keras_output = model.predict(specific_input_data)
print("Keras Model Output (Probabilities):", keras_output)

# Get the predicted class for the Keras model output
keras_predicted_class = np.argmax(keras_output)
print("Keras Model Predicted Class:", keras_predicted_class)
print("Keras Model Predicted Forest Type:", label_encoder.inverse_transform([keras_predicted_class])[0])

# Set the input for the TFLite interpreter
interpreter.set_tensor(input_details[0]['index'], specific_input_data)

# Invoke the TFLite interpreter
interpreter.invoke()

# Get the output from the TFLite model
tflite_output = interpreter.get_tensor(output_details[0]['index'])[0]
print("TFLite Model Output (Probabilities):", tflite_output)

# Get the predicted class for the TFLite model output
tflite_predicted_class = np.argmax(tflite_output)
print("TFLite Model Predicted Class:", tflite_predicted_class)
print("TFLite Model Predicted Forest Type:", label_encoder.inverse_transform([tflite_predicted_class])[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Keras Model Output (Probabilities): [[7.16674782e-04 1.19697856e-04 9.99148369e-01 1.52321336e-05]]
Keras Model Predicted Class: 2
Keras Model Predicted Forest Type: o
TFLite Model Output (Probabilities): [0.         0.         0.99609375 0.        ]
TFLite Model Predicted Class: 2
TFLite Model Predicted Forest Type: o


## 10. Generating the Header File

In [66]:
# Function to convert some hex values into an array for C programming
def hex_to_c_array(hex_data, var_name):
    c_str = ""

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += "#define " + var_name.upper() + '_H\n\n'

    c_str += "/*\n Author: Mouli Sankaran \n"
    c_str += " CAUTION: This is an auto generated file.\n DO NOT EDIT OR MAKE ANY CHANGES TO it.\n"

    # Time stamping of this model data in the generated file
    localtime = time.asctime(time.localtime(time.time()))
    c_str += " This model data was generated on " + localtime + '\n\n'
    print("This model data was generated on:", localtime)

    # Add information about the versions of tools and packages used in generating this header file
    c_str += " Tools used:\n Python: " + str(sys.version) + "\n Numpy: " + str(np.version.version) + \
             "\n TensorFlow: " + str(tf.__version__) + "\n Keras: " + str(tf.keras.__version__) + "\n\n"
    print("Tools used: Python:", sys.version, "\n Numpy:", np.version.version, \
          "\n TensorFlow:", tf.__version__, "\n Keras: ", tf.keras.__version__, "\n\n")

    # Training details of the model
    c_str += ' Model details are:\n'
    c_str += ' NUM_OF_EPOCHS = ' + str(NUM_OF_EPOCHS) + '\n'
    c_str += ' BATCH_SIZE    = ' + str(BATCH_SIZE) + '\n*/\n'
    
    # Generate 'C' constants for the no. of nodes in each layer
    c_str += '\nconst int ' + 'DENSE1_SIZE' + ' = ' + str(DENSE1_SIZE) + ';\n'
    c_str += 'const int ' + 'DENSE2_SIZE' + ' = ' + str(DENSE2_SIZE) + ';\n'      
    
    # Add array length at the top of the file
    c_str += '\nconst unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable
    c_str += 'alignas(8) const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data):
        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n'
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n' + format(''.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'

    return c_str

# Assuming tflite_model is your TensorFlow Lite model
# Write TFLite model to a C source (or header) file
with open("forest_model_esp32.h", 'w') as file:
    file.write(hex_to_c_array(tflite_model, "forest_model_esp32"))


This model data was generated on: Tue Oct 29 22:38:08 2024
Tools used: Python: 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)] 
 Numpy: 1.26.4 
 TensorFlow: 2.17.0 
 Keras:  3.5.0 


